In [1]:
#Finds matches for search term in a page
from bs4 import BeautifulSoup
import requests
import re
import sys
import urllib
from urllib.request import Request, urlopen

def search_url(url, search_term):
  
  # print(url.split("www.")[-1].split("//")[-1].split(".")[0])
  # html = urllib.urlopen(url).read()
  #print (url)
  link=url
  if sys.version_info[0] == 3:
      from urllib.request import urlopen
  else:
      # Not Python 3 - today, it is most likely to be Python 2
      # But note that this might need an update when Python 4
      # might be around one day
      from urllib import urlopen

  try:  
    url=Request(url, headers={'User-Agent': 'Mozilla/5.0}'})
    # Your code where you can use urlopen
    with urlopen(url) as url:
        s = url.read()
  except:
    return 0
  # print(s)

  soup = BeautifulSoup(s, 'html.parser')

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)

  # print(text)
  search_term=search_term.lower()
  #text=text.lower()
  matches=0
  full_extract=""
  matched_sentences=[]

  for line in text.split("\n"):
    if len(line.split(" "))>20:
      for sentence in line.split('.'):
        for match in [m.start() for m in re.finditer(search_term, sentence.lower())]:
            #print (sentence+".")
            matched_sentences.append(sentence+".")
            #print()
            matches=matches+1        
        
  print("Matches in this url - "+str(matches))   
  return({
      "url":link,
      "matches":matches,
      "matched_sentences":matched_sentences
  })

# url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08"
# search_term="Ladakh"
# search_url(url, search_term)

In [2]:
#Reruens list of URLs found in a page
def get_urls(url):
    all_url_count=0
    url_count=0
    url_list=[]

    try:
        r  = requests.get(url)
        data = r.text
        soup = BeautifulSoup(data)    

        for link in soup.find_all('a'):
            all_url_count=all_url_count+1
        #[(all_url_count=all_url_count+1) for link in soup.find_all('a')]
        print("All URL count is "+str(all_url_count))

        # regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
        regex=r'^(http|https|www)'
        for link in soup.find_all('a'):
            url=link.get('href')
            if (url) and bool(re.search(regex, url)):
                #print(url)
                url_list.append(url)
                url_count=url_count+1
            else:
                #print("This was NOT COUNTED "+str(url))
                pass
    except:
        pass
        
            
    print("Filtered URL count is "+str(url_count))
    
    url_list = list(dict.fromkeys(url_list))    
    #print(url_list)
    print("Duplicates removed URL count is "+str(len(url_list)))
    return(url_list)

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime
start_time=datetime.datetime.now()
print(start_time)

# url = raw_input("Enter a website to extract the URL's from: ")
# url="https://medium.com/@dmahugh_70618/deploying-a-flask-app-to-google-app-engine-faa883b5ffab"
# url="https://matadornetwork.com/notebook/how-to-travel-by-cargo-ship/"
# url="https://www.thehindubusinessline.com/opinion/columns/r-srinivasan/india-incs-living-in-denial/article32278690.ece?__sta=vhg.uosvpxezzlbsksaq%7CFYU&__stm_medium=email&__stm_source=smartech&utm_source=ThePrint&utm_medium=Email&utm_campaign=TFPMailer&utm_content=8Aug2020"

# source_url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08" 
# search_term="ladakh"

# source_url="https://en.wikipedia.org/wiki/Tom_Crean_(explorer)"
# search_term="antarctica"

source_url="https://en.wikipedia.org/wiki/Central_Intelligence_Agency"
search_term="cuba"

# source_url="https://science.sciencemag.org/content/359/6380/1146"
# search_term="virality"


depth=2
print("Depth set as "+str(depth))
articles_searched=0
links_parsed=[]

def search_driver(source_url, level):
    level=level+1
    if level<=depth:
        print("    "*level+"LEVEL is "+str(level))    
        print("SOURCE URL IS "+source_url)

        match_dict=search_url(source_url, search_term)
        if match_dict!=0 and match_dict["matches"]!=0:
            print(match_dict)    
        if (level+1<=depth): #So that we don't waste time extracting URLs that are beyond required depth
            for url in get_urls(source_url):
                global links_parsed
                if url not in links_parsed: #Remove lnks that have already been parsed earlier
                    #print("in loop level is "+str(level)+" "+url)
                    links_parsed.append(url)
                    global articles_searched
                    articles_searched=articles_searched+1
                    print()
                    search_driver(url,level)

                    
search_driver(source_url, -1)
print("\nArticles_searched "+str(articles_searched))
print("Articles_searched "+str(len(links_parsed)))

end_time=datetime.datetime.now()
print(end_time)
print("TIME taken in seconds is "+str(end_time-start_time))

2020-08-10 01:59:13.510691
Depth set as 2
LEVEL is 0
SOURCE URL IS https://en.wikipedia.org/wiki/Central_Intelligence_Agency
Matches in this url - 10
{'url': 'https://en.wikipedia.org/wiki/Central_Intelligence_Agency', 'matches': 10, 'matched_sentences': ['In the election of Patrice Lumumba, and his acceptance of Soviet support the CIA saw another possible Cuba.', 'The Bay of Pigs Invasion was a failed military invasion of Cuba undertaken by the CIA-sponsored paramilitary group Brigade 2506 on April 17, 1961.', ' Launched from Guatemala, the invading force was defeated within three days by the Cuban Revolutionary Armed Forces, under the direct command of Prime Minister Fidel Castro.', ' The CIA proceeded to organize the operation with the aid of various Cuban counter-revolutionary forces, training Brigade 2506 in Guatemala.', ' Over 1,400 paramilitaries set out for Cuba by boat on April 13.', ' Two days later on April 15, eight CIA-supplied B-26 bombers attacked Cuban airfields.', ' Th

Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.wikidata.org/wiki/Special:EntityPage/Q37230
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://stats.wikimedia.org/#/en.wikipedia.org
Matches in this url - 0

    LEVEL is 1
SOURCE URL IS http://www.cia.gov
Matches in this url - 0
All URL count is 229
Filtered URL count is 18
Duplicates removed URL count is 14

        LEVEL is 2
SOURCE URL IS https://youtu.be/KvG8c8aVtl8
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://twitter.com/CIA
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.facebook.com/Central.Intelligence.Agency
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.youtube.com/user/ciagov
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.flickr.com/photos/ciagov
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.youtube.com/user/ciagov
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://

Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://subscribe.washingtonpost.com/acq?promo=o2_gift_wpfd20&destination=https://www.washingtonpost.com/world/national-security/black-budget-summary-details-us-spy-networks-successes-failures-and-objectives/2013/08/29/7e57bb78-10ab-11e3-8cdd-bcdc09410972_story.html&tid=nav_giftsub_logged_out
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://subscribe.washingtonpost.com/acquisition/?nid=top_pb_subscribe&promo=o3&oscode=RPWH&acqEntType=mktg_onsite_article&destination=https://www.washingtonpost.com/world/national-security/black-budget-summary-details-us-spy-networks-successes-failures-and-objectives/2013/08/29/7e57bb78-10ab-11e3-8cdd-bcdc09410972_story.html&tid=nav_subscribe_logged_out
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.washingtonpost.com/subscribe/signin/?tid=nav_sign_in
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.washingtonpost.com/my-post/#/?tid=nav_ac

Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.fas.org/siteindx.html
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.fas.org/search.html
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.fas.org/join.html
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.fas.org/sgp/foia/hogan.html
Matches in this url - 0

    LEVEL is 1
SOURCE URL IS https://fas.org/blog/secrecy/2011/10/brennan_ctr_report.html
Matches in this url - 0
All URL count is 76
Filtered URL count is 58
Duplicates removed URL count is 43

        LEVEL is 2
SOURCE URL IS https://www.facebook.com/fascientists
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://twitter.com/FAScientists
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.linkedin.com/groups/Federation-American-Scientists-FAS-3024349
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://fas.org/about-fas/
Matches in this url - 0

        LEV

Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/travel
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/health
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/home
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/opinion
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-srv/weather
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/weekly

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/digest
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/admin/classifieds
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-dyn/print
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.washingtonpost.com/wp-adv/a

...

Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://books.google.co.in/books?id=F2nSTwbFAh0C&lpg=PA121&pg=PA119&redir_esc=y&hl=en&output=html_text
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.com/accounts/Login?service=print&continue=https://books.google.co.in/books%3Fop%3Dlookup%26id%3DF2nSTwbFAh0C%26continue%3Dhttps://books.google.co.in/books%253Fid%253DF2nSTwbFAh0C%2526lpg%253DPA121%2526pg%253DPA119%2526redir_esc%253Dy%2526hl%253Den&hl=en
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://books.google.co.in/books?id=F2nSTwbFAh0C&sitesec=buy&source=gbs_atb
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://books.rediff.com/book/ISBN:0061147788
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://www.flipkart.com/search.php?query=0061147788

        LEVEL is 2
SOURCE URL IS http://www.infibeam.com/Books/search?q=9780061147784
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://books.go

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.co.in/search?tbo=p&tbm=bks&q=inauthor:%22George+Tenet%22&source=gbs_metadata_r&cad=4
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.co.in/search?tbo=p&tbm=bks&q=inauthor:%22Bill+Harlow%22&source=gbs_metadata_r&cad=4
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.co.in/search?tbo=p&tbm=bks&q=subject:%22Biography+%26+Autobiography%22
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.co.in/search?tbo=p&tbm=bks&q=subject:%22Biography+%26+Autobiography+Political%22
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.co.in/search?tbo=p&tbm=bks&q=subject:%22Biography+%26+Autobiography+/+Political%22&source=gbs_metadata_r&cad=4
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.co.in/search?tbo=p&tbm=bks&q=subject:%22Political+Science+/+General%22&source=gbs_metadata_r&cad=4
Matches

Matches in this url - 0

        LEVEL is 2
SOURCE URL IS http://books.google.co.in/support/answer/180577?hl=en-IN&url=https://books.google.co.in/books?id=F2nSTwbFAh0C&pg=PA121&q=%22Afghan+sanctuary%22&redir_esc=y&hl=en&v=F2nSTwbFAh0C&is=atb
Matches in this url - 0

    LEVEL is 1
SOURCE URL IS https://books.google.com/books?id=F2nSTwbFAh0C&pg=PA177
Matches in this url - 0
All URL count is 72
Filtered URL count is 58
Duplicates removed URL count is 47

        LEVEL is 2
SOURCE URL IS https://www.google.com/accounts/Login?service=print&continue=https://books.google.co.in/books%3Fid%3DF2nSTwbFAh0C%26pg%3DPA177%26redir_esc%3Dy%26hl%3Den&hl=en
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://books.google.co.in/books?id=F2nSTwbFAh0C&pg=PA177&redir_esc=y&hl=en&output=html_text
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://www.google.com/accounts/Login?service=print&continue=https://books.google.co.in/books%3Fop%3Dlookup%26id%3DF2nSTwbFAh0C%26continue%3Dh

Matches in this url - 0
All URL count is 160
Filtered URL count is 153
Duplicates removed URL count is 118

        LEVEL is 2
SOURCE URL IS https://www.theguardian.com/international
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://support.theguardian.com/contribute?INTCMP=header_support_contribute&acquisitionData=%7B%22componentType%22:%22ACQUISITIONS_HEADER%22,%22componentId%22:%22header_support_contribute%22,%22source%22:%22GUARDIAN_WEB%22%7D
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://support.theguardian.com/subscribe?INTCMP=header_support_subscribe&acquisitionData=%7B%22componentType%22:%22ACQUISITIONS_HEADER%22,%22componentId%22:%22header_support_subscribe%22,%22source%22:%22GUARDIAN_WEB%22%7D
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://jobs.theguardian.com/?INTCMP=jobs_int_web_newheader
Matches in this url - 0

        LEVEL is 2
SOURCE URL IS https://profile.theguardian.com/signin?INTCMP=DOTCOM_NEWHEADER_SIGNIN&compon

## OBSERVATIONS
### For IndiaToday
Around 15-20% URLs repeated on the page itself
source_url="https://www.indiatoday.in/india/story/india-china-military-level-talks-ladakh-depsang-situation-1709067-2020-08-08" 
search_term="ladakh"
* Depth=1, Time:40sec, Articles_searched:68;
* Depth=2, Time:47min, Articles_searched:3933(=68+3865);
* Depth=2, Time:40sec, Articles_searched:, With repetition check;

### For Wikipedia
source_url="https://en.wikipedia.org/wiki/Tom_Crean_(explorer)"
search_term="antarctica"
* Depth=1, Time:2min, Articles_searched:52;
* Depth=2, Time:min, Articles_searched:;

source_url="https://en.wikipedia.org/wiki/Central_Intelligence_Agency"
search_term="cuba"
Around 90% URLs repeated on the page itself
* Depth=1, Time16.5min:, Articles_searched:353;
* Depth=2, Time:, Articles_searched:;

### For Research paper
Around 85% URLs repeated on the page itself

source_url="https://en.wikipedia.org/wiki/Tom_Crean_(explorer)"
search_term="antarctica"
* Depth=1, Time:2.5min, Articles_searched:55;
* Depth=2, Time:1h20min, Articles_searched:684;

